In [ ]:
import import_ipynb
import environment.support as support
import environment.strings as strings

In [ ]:
class Room:
    def __init__(self, messages = dict(), people = dict(), items = dict(), interactions = dict(), isInventory = False, isItem = False, isRepeat = False):
        self.messages = messages
        self.people = people
        self.items = items
        self.interactions = interactions
        self.isInventory = isInventory
        self.isItem = isItem
        self.isRepeat = isRepeat
    
    def runRoom(self):
        if not self.isInventory and not self.isItem:
            if not self.isRepeat:
                support.writeConsole(self.messages["enterText"]) #"enterText" reserved for room entry
            print()
            support.writeConsole(self.messages["name"]) #"name" reserved for vanity area name
        peopleMessages = [person["enterText"] for person in self.people.values() if "enterText" in person]
        itemMessages = [item["enterText"] for item in self.items.values() if "enterText" in item]
        interactionMessages = [interact["enterText"] for interact in self.interactions.values() if "enterText" in interact]
        for person in peopleMessages:
            support.writeConsole(person)
        for message in itemMessages:
            support.writeConsole(message)      
        for message in interactionMessages:
            support.writeConsole(message)
        
        peopleMenu = [person["menuText"] for person in self.people.values() if "menuText" in person] #"menuText reserved for menu text and callback"
        itemMenu = [item["menuText"] for item in self.items.values() if "menuText" in item]
        interactionMenu = [interact["menuText"] for interact in self.interactions.values() if "menuText" in interact]
        menuDict = dict()
        
        for dictionary in interactionMenu:
            for key in dictionary:
                menuDict[key] = dictionary[key]
        for dictionary in peopleMenu:
            for key in dictionary:
                menuDict[key] = dictionary[key]
        for dictionary in itemMenu:
            for key in dictionary:
                menuDict[key] = dictionary[key]
        if self.isItem:
            self.isItem.desc()
            setattr(self, "isItem", eval("strings." + self.isItem.attrName))
            for action in self.isItem["actions"]:
                menuDict[action] = self.isItem["actions"][action]
            menuDict["Never mind"] = lambda : None
        if self.isInventory:
            for item in self.isInventory:
                menuDict[item] = lambda item=item: Room(isItem = self.isInventory[item]).runRoom()
            menuDict["Never mind"] = lambda : None
        if not self.isInventory and not self.isItem:
            if support.Inventory.getInventory():
                menuDict["Check pockets"] = lambda : Room(isInventory = support.Inventory.getInventory()).runRoom()
        
        roomMenu = Menu(menuDict)
        return roomMenu.runMenu()
    
    def interactItem(self, item, action = None, target = None):
        #TODO
        pass
        
    def interactPerson(self, person, action = None):
        #TODO
        pass
        
    def interactRoom(self, action, target = None):
        #TODO
        pass
    
    def getMessages(self):
        return self.messages
    
    def getPeople(self):
        return self.people
    
    def getItems(self):
        return self.items
    
    def getInteractions(self):
        return self.interactions
    
    def addMessage(self, key, message):
        self.messages[key] = message
    
    def addPerson(self, key, person):
        self.people[key] = person
    
    def addItem(self, key, item):
        self.items[key] = item
    
    def addInteraction(self, key, interaction):
        self.interactions[key] = interaction
        
    def removeMessage(self, key):
        del self.messages[key]
        
    def removePerson(self, key):
        del self.people[key]
    
    def removeItem(self, key):
        del self.items[key]
        
    def removeInteraction(self, key):
        del self.interactions[key]

In [2]:
class Menu:
    def __init__(self, options = dict()):
        self.options = options
        
    def runMenu(self, consoleChar = "~"):
        listed = list(zip(self.options, range(len(self.options))))
        for key, i in listed:
            support.writeConsole(f"{i + 1} - {key}")
        selection = support.getIntInput(1, len(self.options), consoleChar)
        return self.options[listed[selection - 1][0]]()
        
    def getOptions(self):
        return self.options
        
    def addOption(self, key, option):
        self.options[key] = option